#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

In [0]:
%sql
create catalog if not exists enterprise_fleet_analytics;
create database if not exists enterprise_fleet_analytics.landing_zone;
create volume if not exists enterprise_fleet_analytics.landing_zone.landing_volume;

In [0]:
'''By Manually seeing the source files, able to see following patterns

1. Source1
- There are five columns namely Shipment_id, first_name, last_name, age, role
- source file contains only 5 series shipment ids
- shipment id 5000004 is not having lastname
- shipment id 5000006 is having age as string value and have an extra column with value as Additionalcolumn
- shipment id 5000007 have no role in it
- shipment id 5010014 have no last name 
- row number 33 has no vlaues populated
- shipment id 5010049 have no other fields populated
- row number 51 is having "ten" as shipment_id

2. Source2
- There are 7 columns namely Shipment_id, first name, last name, age, role, location, vehicle type
- source file contains only 6 series shipment ids
- shipment id 6000003 is not haivng vehicle type 
- shipment id 6000004 is not having last_name
- shipment id 6000006 is having age in string
- shipment id 6000007 is not having role
- shipment id 6000009 is not having first name
- shipment id 6000014 is not having last name
- shipment id 6000015 is not having age
- shipment id 6000026 is not having vehicle type
- row number 33 is not having values
- shipment id 6000038 is not having vehicle type
- shipment id 6000040 is not having vehicle type
- shipment id 6000045 is not having vehicle type
- shipemnt id 6000049 is not having anyother values
- shipment id 6000052 is not having vehicle type
- shipment id 6000056 is not having vehicle type
- shipment id 6000064 is not having role and vehicle type
- shipment id 6000069 is not having vehicle type

3. Logistic_shipment_detail
- json file
- There are 11 columns namely Shipment_id, Order_id, source_city, destination_city, shipment_status, cargo_type, vehicle_type, payment_mode, shipment_weight_kg, shipment_cost, shipment_date

4. Master_city_list
- There are 4 columns namely city_name, country, latitude, longitude
'''

####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType,DateType,ArrayType,MapType
from pyspark.sql.functions import col,lit,initcap
#Source 1
source1_schema = StructType([
    StructField("shipment_id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("role", StringType(), True)
])

source1_df = spark.read.csv("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_source1",header=True,inferSchema=True).toDF("shipment_id","first_name","last_name","age","role")
source1_df.printSchema()
source1_df.show(5)
source1_df.describe().show()
print(f"Total number of records {source1_df.count()}")
print(f"Total number of duplicate records {source1_df.count()-source1_df.distinct().count()}\n")

#Source 2
source2_schema = StructType([
    StructField("shipment_id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("role", StringType(), True),
    StructField("hub_location", StringType(), True),
    StructField("vehicle_type", StringType(), True)
])

source2_df = spark.read.csv("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_source2",header=True,inferSchema=True).toDF("shipment_id","first_name","last_name","age","role","hub_location","vehicle_type")
source2_df.printSchema()
source2_df.show(5)
source2_df.summary().show()
#source2_df.describe().show()
print(f"Total number of records {source2_df.count()}")
print(f"Total number of duplicate records {source2_df.count()-source2_df.distinct().count()}\n")

#source 3
logistic_shipment_detail_schema = StructType([
    StructField("shipment_id", IntegerType(), True),
    StructField("order_id", StringType(), True),
    StructField("source_city", StringType(), True),
    StructField("destination_city", StringType(), True),
    StructField("shipment_status", StringType(), True),
    StructField("cargo_type", StringType(), True),
    StructField("vehicle_type", StringType(), True),
    StructField("payment_mode", StringType(), True),
    StructField("shipment_weight_kg", FloatType(), True),    
    StructField("shipment_cost", FloatType(), True),
    StructField("shipment_date", StringType(),True)
])
logistic_shipment_detail_df = spark.read.schema(logistic_shipment_detail_schema).json("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_shipment_detail_3000.json",multiLine=True)
logistic_shipment_detail_df.printSchema()
logistic_shipment_detail_df.show(5)
logistic_shipment_detail_df.summary()
#logistic_shipment_detail_df.filter(logistic_shipment_detail_df["_corrupt_record"].isNotNull()).show(truncate=False)
print(f"Total number of records {logistic_shipment_detail_df.count()}")
print(f"Total number of duplicate records {logistic_shipment_detail_df.count()-logistic_shipment_detail_df.distinct().count()}\n")

#source 4
master_city_list_schema = StructType([
    StructField("city_name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True)
])
master_city_list_df = spark.read.schema(master_city_list_schema).csv("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/Master_City_List.csv",header=True,inferSchema=False)
master_city_list_df.printSchema()
master_city_list_df.show(5)
master_city_list_df.summary()
print(f"Total number of records {source2_df.count()}")
print(f"Total number of duplicate records {source2_df.count()-source2_df.distinct().count()}\n")


###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:
3. fewer columns than expected
4. more columns than expected

In [0]:
#Create a Spark Session Object
#from spark.session import SparkSession
#spark = SparkSession.builder.appName("Logistics").getOrCreate()

source1_df.where("shipment_id rlike '[^0-9]'" or "age rlike '[^0-9]'").show()
source2_df.where("shipment_id rlike '[^0-9]'" or "age rlike '[^0-9]'").show()

###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
from pyspark.sql.functions import lit,col,replace,regexp_replace
from pyspark.sql.types import IntegerType
source1_df=spark.read.csv("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_source1",header=True,inferSchema=True)
filtereds1df=source1_df.dropna(subset=["shipment_id"])
formatteds1df = filtereds1df.withColumn("data_source",lit("system1"))
formatteds1df = formatteds1df.withColumn("shipment_id",regexp_replace(col("shipment_id"),"ten","10").cast(IntegerType()))
#formatteds1df.where("shipment_id in ('10')").show()
formatteds1df.printSchema()
formatteds1df.show(5)
#source1_df.show(5)

source2_df=spark.read.csv("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_source2",header=True,inferSchema=True)
formatteds2df = source2_df.withColumn("data_source",lit("system2"))
formatteds2df.printSchema()
formatteds2df.show(5)

#merging both source files into single canonical schema
rawdf_merged = formatteds1df.unionByName(formatteds2df,allowMissingColumns=True)
display(rawdf_merged)
rawdf_merged.printSchema()


#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
#Cleansing (removela of unwanted datasets)
cleaned_df = rawdf_merged.dropna(subset=["shipment_id","role"])
cleaned_df = cleaned_df.na.drop(how="all",subset=["first_name","last_name"])
cleaned_df.printSchema()

In [0]:
from pyspark.sql.functions import regexp_replace
#Scrubbing (converting Raw to Tidy)
#4.Age Defaulting Rule - Fill NULL values in the age column with: -1
scrubbeddf = cleaned_df.na.fill("-1",subset=['age'])

#5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN
scrubbeddf = scrubbeddf.na.fill("UNKNOWN",subset=['vehicle_type'])

#6. Invalid Age Replacement - Replace the following values in age: "ten" to -1 "" to -1
scrubbeddf = scrubbeddf.withColumn("age",regexp_replace("age","ten|''","-1"))

#7. Vehicle Type Normalization - Replace inconsistent vehicle types: truck to LMV bike to TwoWheeler
replace_val = {'truck':'LMV','bike':'TwoWheeler'}
scrubbeddf = scrubbeddf.replace(replace_val,'vehicle_type')

####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

Creating shipments Details data Dataframe creation <br>
1. Create a DF by Reading Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

In [0]:
logistic_shipment_detail_schema = StructType([
    StructField("shipment_id", IntegerType(), True),
    StructField("order_id", StringType(), True),
    StructField("source_city", StringType(), True),
    StructField("destination_city", StringType(), True),
    StructField("shipment_status", StringType(), True),
    StructField("cargo_type", StringType(), True),
    StructField("vehicle_type", StringType(), True),
    StructField("payment_mode", StringType(), True),
    StructField("shipment_weight_kg", FloatType(), True),    
    StructField("shipment_cost", FloatType(), True),
    StructField("shipment_date", StringType(),True)
])

logistic_shipment_detail_df = spark.read.schema(logistic_shipment_detail_schema).json("/Volumes/enterprise_fleet_analytics/landing_zone/landing_volume/logistics_shipment_detail_3000.json",multiLine=True)
logistic_shipment_detail_df.printSchema()
logistic_shipment_detail_df.show(5)
print(f"Total number of records {logistic_shipment_detail_df.count()}")
print(f"Total number of duplicate records {logistic_shipment_detail_df.count()-logistic_shipment_detail_df.distinct().count()}\n")

Standardizations:<br>

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: DF of merged(logistics_source1 & logistics_source2) <br>
age: Cast String to Integer<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
#Add a column
#Source File: DF of logistics_shipment_detail_3000.json : domain as 'Logistics', current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
from pyspark.sql.functions import current_timestamp,lit
logistic_shipment_detail_df = logistic_shipment_detail_df.withColumn("domain",lit("Logistics"))
logistic_shipment_detail_df = logistic_shipment_detail_df.withColumn("ingestion_timestamp",current_timestamp())
logistic_shipment_detail_df = logistic_shipment_detail_df.withColumn("is_expedited",lit("False"))
display(logistic_shipment_detail_df.take(2))

In [0]:
'''Column Uniformity: 
role - Convert to lowercase
Source File: DF of merged(logistics_source1 & logistics_source2)
vehicle_type - Convert values to UPPERCASE
Source Files: DF of logistics_shipment_detail_3000.json 
hub_location - Convert values to initcap case
Source Files: DF of merged(logistics_source1 & logistics_source2)'''
from pyspark.sql.functions import initcap, lower, upper
uniformed_df1 = scrubbeddf.withColumn("role",lower(col("role")))
uniformed_df2 = logistic_shipment_detail_df.withColumn("vehicle_type",upper(col("vehicle_type")))
uniformed_df1 = uniformed_df1.withColumn("hub_location",initcap(col("hub_location")))
display(uniformed_df1.limit(2))
display(uniformed_df2.limit(2))

In [0]:
'''
Format Standardization:
Source Files: DF of logistics_shipment_detail_3000.json
Convert shipment_date to yyyy-MM-dd
Ensure shipment_cost has 2 decimal precision 
'''
from pyspark.sql.functions import to_date, round
from pyspark.sql.types import DecimalType
formatteddf2 = uniformed_df2.withColumn("shipment_date",to_date(col("shipment_date"),"yy-MM-dd"))
formatteddf2 = formatteddf2.withColumn("shipment_cost",round(col("shipment_cost"),2).cast(DecimalType(10,2)))
display(formatteddf2.limit(2))

#seen a display issue when given .take(2). As, take is a spark action, it converts the Spark rows to python objects, so DecimalType becomes 
# python Decimal. Python prints decimals with their full internal scale 

In [0]:
'''
Data Type Standardization
Standardizing column data types to fix schema drift and enable mathematical operations.
Source File: DF of merged(logistics_source1 & logistics_source2)
age: Cast String to Integer
Source File: DF of logistics_shipment_detail_3000.json
shipment_weight_kg: Cast to Double
Source File: DF of logistics_shipment_detail_3000.json
is_expedited: Cast to Boolean
'''
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType
uniformed_df1.printSchema()
standarizeddf1 = uniformed_df1.withColumn("age",col("age").cast(IntegerType()))

standarizeddf2 = formatteddf2.withColumn("shipment_weight_kg",col("shipment_weight_kg").cast(DoubleType()))
standarizeddf2 = standarizeddf2.withColumn("is_expedited",col("is_expedited").cast(BooleanType()))

standarizeddf1.printSchema()
standarizeddf2.printSchema()

In [0]:
'''
Naming Standardization
Source File: DF of merged(logistics_source1 & logistics_source2)
Rename: first_name to staff_first_name
Rename: last_name to staff_last_name
Rename: hub_location to origin_hub_city
'''
standarizeddf1 = standarizeddf1.withColumnsRenamed({'first_name':'staff_first_name','last_name':'staff_last_name','hub_location':'origin_hub_city'})

In [0]:
'''
Reordering columns logically in a better standard format:
Source File: DF of Data from all 3 files
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)
'''

Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

**4. Flag shipment status (`is_expedited`)**
* **Scenario:** The Operations team needs to track shipments is IN_TRANSIT or DELIVERED.
* **Action:** Flag as **'True'** if the `shipment_status` IN_TRANSIT or DELIVERED.

###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: DF of logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: DF of json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: DF of json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: DF of json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: DF of json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending).

**7. Limit (Top-N Analysis)**<br>
Source File: DF of json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> DF of logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> DF of logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF) and Master_City_List.csv<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the dataframe of corporate `Master_City_List.csv`.
* **Action:** Compare values against this Master_City_List list.

### **3. Lookup & Enrichment**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF) and Master_City_List.csv dataframe<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` (eg. "Pune") in a Master Latitude/Longitude Master_City_List.csv dataframe and enrich our logistics_source (merged dataframe) by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: DF of All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
DF of logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.